In [13]:
import cv2
import imutils
import numpy as np
import random

In [14]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [15]:
img = cv2.imread('...')
googly_eye = cv2.imread('./googly_eye_tr.png', cv2.IMREAD_UNCHANGED)

In [17]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [18]:
random.seed(125)

In [19]:
faces = face_cascade.detectMultiScale(gray, 1.3, 8)
for (x,y,w,h) in faces:
    #img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray, 1.3, 5)
    for (ex,ey,ew,eh) in eyes:
        eye = cv2.resize(googly_eye.copy(), (ew, eh))
        eye = imutils.rotate(eye, int(360*random.random()))
        rows,cols,channels = eye.shape
        b,g,r,a = cv2.split(eye)
        overlay_color = cv2.merge((b,g,r))
        
        mask = cv2.medianBlur(a,5)
        h, w, _ = overlay_color.shape
        
        img_eye_roi = roi_color[ey:ey+cols, ex:ex+rows]
        
        img1_bg = cv2.bitwise_and(img_eye_roi.copy(),img_eye_roi.copy(),mask = cv2.bitwise_not(mask))
        img2_fg = cv2.bitwise_and(overlay_color,overlay_color,mask = mask)
        
        roi_color[ey:ey+ew, ex:ex+eh] = cv2.add(img1_bg, img2_fg)
        #cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)


In [ ]:
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
#https://gist.github.com/clungzta/b4bbb3e2aa0490b0cfcbc042184b0b4e
#https://stackoverflow.com/questions/40895785/using-opencv-to-overlay-transparent-image-onto-another-image
#https://github.com/fatcat2/googly-eyes/blob/master/Python/piceyefilters.py